  # Apply encoder to morphological profiles to get latent space representations :

# Setup

In [3]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import os
import sys
import csv
import json
import time
import types, copy, pprint
import logging 
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)
# import shutil
# import getpass
import yaml

import hydra 
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf


from matplotlib import pyplot as plt
from typing import Dict, List, Tuple
from scipy.sparse import csr_matrix
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy  
import torch
import torch.nn.functional as F
from torchinfo import summary
import wandb

torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
 

pp = pprint.PrettyPrinter(indent=4)
pd.options.display.width = 132
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

os.environ["WANDB_NOTEBOOK_NAME"] = "AE-MAIN-SNNL.ipynb"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

insert ./src
insert ../..
['../..', './src', '/home/kevin/miniforge3/envs/ptsnnl/lib/python311.zip', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages/cytominer_eval-0.1-py3.11.egg']


In [5]:
import snnl.utils as utils
from snnl.utils import parse_args, load_configuration, set_global_seed, get_device, set_device
from snnl.utils import display_model_summary, define_autoencoder_model
from KevinsRoutines.utils.utils_general  import  list_namespace


In [6]:
import sklearn.metrics as skm 
import scipy.stats as sps 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform, euclidean
import matplotlib.pyplot as plt
import seaborn as sb

# from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, classification_report, precision_recall_fscore_support
# from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, roc_curve, RocCurveDisplay
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [7]:
timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
logger = logging.getLogger(__name__) 
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(name)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)

logger.info(f" Excution started : {timestamp} ")
logger.info(f" Pytorch version  : {torch.__version__}")
logger.info(f" Scipy version    : {scipy.__version__}  \t\t Numpy version : {np.__version__}")
logger.info(f" WandB version    : {wandb.__version__}  \t\t Pandas version: {pd.__version__}  ")


2024-08-28 17:38:11,831 - __main__ - INFO: -  Excution started : 2024_08_28_17:38:11 
2024-08-28 17:38:11,832 - __main__ - INFO: -  Pytorch version  : 2.2.2
2024-08-28 17:38:11,833 - __main__ - INFO: -  Scipy version    : 1.12.0  		 Numpy version : 1.26.4
2024-08-28 17:38:11,834 - __main__ - INFO: -  WandB version    : 0.17.4  		 Pandas version: 2.2.1  


### main(args)

In [10]:
RUNMODE       = "baseline" 
# RUNMODE       = "snnl"
WANDB_ACTIVE  = False
LATENT_DIM    = 150
HIDDEN_1      = 512
EPOCHS        = 5
GPU_ID        = 1
COMPOUNDS_PER_BATCH = 200
# LEARNING_RATE = 1.00e-04
# TEMP          = 0.05
# TEMP_LR       = 1.00e-05
CONFIGURATION = "./hyperparameters/ae_baseline_150_512_cpb.yaml"
# CONFIGURATION   = "./hyperparameters/ae_sn_150_512_cpb.yaml"
OUTPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/" 

In [12]:
cli_args = f" --runmode            {RUNMODE} " \
            f" --configuration      {CONFIGURATION}" \
            f" --cpb                {COMPOUNDS_PER_BATCH}" \
            f" --epochs             {EPOCHS} " \
            f" --seed               4321" \
            f" --gpu_id             {GPU_ID}  "  \
            f" "
cli_args = utils.parse_args(cli_args.split())
cli_args

args = utils.load_configuration(cli_args)
args.ckpt
args


Namespace(configuration='./hyperparameters/ae_baseline_150_512_cpb.yaml', ckpt=None, cpb=200, exp_title=None, epochs=5, gpu_id=1, learning_rate=None, exp_id=None, runmode='baseline', random_seed=4321, use_prim_optimizer=True, use_temp_optimizer=False, use_annealing=False, use_single_loss=False, temperature=None, temperatureLR=None, wandb=False)

2024-08-28 17:40:07,967 - snnl.utils.utils_ptsnnl - INFO: -  command line param configuration             : [./hyperparameters/ae_baseline_150_512_cpb.yaml]
2024-08-28 17:40:07,967 - snnl.utils.utils_ptsnnl - INFO: -  command line param ckpt                      : [None]
2024-08-28 17:40:07,968 - snnl.utils.utils_ptsnnl - INFO: -  command line param cpb                       : [200]
2024-08-28 17:40:07,968 - snnl.utils.utils_ptsnnl - INFO: -  command line param exp_title                 : [None]
2024-08-28 17:40:07,969 - snnl.utils.utils_ptsnnl - INFO: -  command line param epochs                    : [5]
2024-08-28 17:40:07,970 - snnl.utils.utils_ptsnnl - INFO: -  command line param gpu_id                    : [1]
2024-08-28 17:40:07,970 - snnl.utils.utils_ptsnnl - INFO: -  command line param learning_rate             : [None]
2024-08-28 17:40:07,972 - snnl.utils.utils_ptsnnl - INFO: -  command line param exp_id                    : [None]
2024-08-28 17:40:07,973 - snnl.utils.utils_pt

namespace(project_name='CellPainting_Profiles',
          exp_id=None,
          exp_name_pfx=None,
          exp_name='AE_20240828_1740',
          exp_date='20240828_1740',
          exp_title='scpb200_150Ltnt_512',
          exp_description='AE baseline - snglOpt, 150 dim latent, 512 Midlayer, 200 cpb',
          exp_folder=None,
          folder_sfx=None,
          random_seed=4321,
          units=[['linear', 1471, 1024],
                 ['relu', 0, 0],
                 ['linear', 1024, 512],
                 ['relu', 0, 0],
                 ['linear', 512, 150],
                 ['linear', 150, 512],
                 ['relu', 0, 0],
                 ['linear', 512, 1024],
                 ['relu', 0, 0],
                 ['linear', 1024, 1471]],
          input_shape=1471,
          code_units=150,
          embedding_layer=4,
          show_every=1,
          save_every=100,
          use_prim_optimizer=True,
          use_prim_scheduler=True,
          learning_rate=0.001,
   

In [13]:
set_global_seed(args.random_seed)
if args.ckpt is not None:
    if os.path.exists(os.path.join('ckpts', args.ckpt)):
        logger.info(f"Checkpoint {args.ckpt} found")
        logger.info(f"Resuming training using checkpoint: {args.ckpt}")
    else:
        logger.error(f"*** Checkpoint {args.ckpt} not found *** \n")
        raise ValueError(f"\n *** Checkpoint DOESNT EXIST *** \n")

if args.gpu_id is not None:
    _ = get_device(verbose=True)
    # print(f"Current device is : {current_device}")    
    args.current_device = set_device(args.gpu_id)
    

 device: 0   Quadro GV100                   :  free: 32,536,854,528 B   (30.30 GB)    total: 34,069,872,640 B   (31.73 GB)
 device: 1   Quadro GV100                   :  free: 33,191,165,952 B   (30.91 GB)    total: 34,069,872,640 B   (31.73 GB)


2024-08-28 17:40:13,683 - snnl.utils.utils_ptsnnl - INFO: -  Current CUDA Device is:  cuda:0 - Quadro GV100
2024-08-28 17:40:13,684 - snnl.utils.utils_ptsnnl - INFO: -  Switched to: Quadro GV100 - 1


 device: 2   NVIDIA TITAN Xp                :  free: 11,996,364,800 B   (11.17 GB)    total: 12,774,539,264 B   (11.90 GB)


## Define dataset and dataloaders

### Load Cell Painting Dataset (All training & test samples)

In [34]:
# args.cellpainting_args['validation_path']

args.cellpainting_args['training_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
args.cellpainting_args['train_start'] = 0
args.cellpainting_args['train_end'] = 312_000  ## 312_000

# # args.cellpainting_args['validation_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'
# # args.cellpainting_args['val_start'] = 0
# # args.cellpainting_args['val_end'] = 312_000

args.cellpainting_args['test_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'
args.cellpainting_args['test_start'] = 0 
args.cellpainting_args['test_end'] = 34_542

args.cellpainting_args

{'sample_size': 3,
 'batch_size': 1,
 'compounds_per_batch': 200,
 'training_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv',
 'validation_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv',
 'test_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv',
 'train_start': 0,
 'train_end': 312000,
 'val_start': 0,
 'val_end': 24000,
 'test_start': 0,
 'test_end': 34542,
 'chunksize': None,
 'conversions': None,
 'iterator': True}

### Load Cell Painting Dataset

In [35]:
data_loader = utils.build_dataloaders(args, data = ['train',  'test'])
data_loader.keys()

2024-08-28 17:53:32,209 - root - INFO: -  load cellpainting
2024-08-28 17:53:32,210 - snnl.utils.dataloader - INFO: -  Building CellPantingDataset for train
2024-08-28 17:53:32,211 - snnl.utils.dataloader - INFO: -  filename:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv
2024-08-28 17:53:32,211 - snnl.utils.dataloader - INFO: -  type    :  train
2024-08-28 17:53:32,212 - snnl.utils.dataloader - INFO: -  start   :  0
2024-08-28 17:53:32,212 - snnl.utils.dataloader - INFO: -  end     :  312000
2024-08-28 17:53:32,213 - snnl.utils.dataloader - INFO: -  numrows :  312000
2024-08-28 17:53:32,213 - snnl.utils.dataloader - INFO: -  names   :  None     usecols :  None
2024-08-28 17:53:32,214 - snnl.utils.dataloader - INFO: -  batch_size  :  1
2024-08-28 17:53:32,215 - snnl.utils.dataloader - INFO: -  sample_size :  3
2024-08-28 17:53:32,215 - snnl.utils.dataloader - INFO: -  compounds_per_batch :  200
2024-08-28 

 load cellpainting
 Dataset size: 312000   rows per batch: 600
 Dataset size: 34542   rows per batch: 600


dict_keys(['train', 'test'])

In [54]:
# idx = 0 
# data_loader_iterator = iter(data_loader['test'])

# Define Autoencoder Model

In [17]:
print(f"   Units             {args.units}")
print(f"   Latent dim        {args.code_units}")
print(f"   loss_factor       {args.loss_factor}")
print(f"   snnl_factor       {args.snnl_factor}")
print(f"   temperature       {args.temperature}")
print(f"   learning_rate     {args.learning_rate}")
print(f"   temperatureLR:    {args.temperatureLR}")

list_namespace(args)

   Units             [['linear', 1471, 1024], ['relu', 0, 0], ['linear', 1024, 512], ['relu', 0, 0], ['linear', 512, 150], ['linear', 150, 512], ['relu', 0, 0], ['linear', 512, 1024], ['relu', 0, 0], ['linear', 1024, 1471]]
   Latent dim        150
   loss_factor       1.0
   snnl_factor       0.0
   temperature       0.0
   learning_rate     0.001
   temperatureLR:    0.0

command line parms : 
-----------------------
SGD_momentum.............  0
SGD_weight_decay.........  0
adam_weight_decay........  0.01
batch_size...............  1

    cellpainting_args   (dict)
    ----------------------------
    batch_size...............  1
    chunksize................  None
    compounds_per_batch......  200
    conversions..............  None
    iterator.................  True
    sample_size..............  3
    test_end.................  33600
    test_path................  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_te

In [18]:
args.runmode
model = define_autoencoder_model(args, verbose=True)
print(args.current_device, model.device)

'baseline'

2024-08-28 17:42:50,156 - snnl.utils.utils_notebook - INFO: - Defining model in baseline mode
2024-08-28 17:42:50,157 - snnl.models.autoencoder - INFO: -     layer pair:    0  type:linear           input:   1471  output:   1024    weights: [1024, 1471]   
2024-08-28 17:42:50,204 - snnl.models.autoencoder - INFO: -     layer pair:    1  type:relu             input:      0  output:      0    weights: [0, 0]   
2024-08-28 17:42:50,205 - snnl.models.autoencoder - INFO: -     layer pair:    2  type:linear           input:   1024  output:    512    weights: [512, 1024]   
2024-08-28 17:42:50,214 - snnl.models.autoencoder - INFO: -     layer pair:    3  type:relu             input:      0  output:      0    weights: [0, 0]   
2024-08-28 17:42:50,215 - snnl.models.autoencoder - INFO: -     layer pair:    4  type:linear           input:    512  output:    150    weights: [150, 512]   
2024-08-28 17:42:50,217 - snnl.models.autoencoder - INFO: -     layer pair:    5  type:linear           input: 

 EMBEDDING LAYER: 4
 Device cuda:1 will be used

------------------------------------------------------------
 Building Base Model from NOTEBOOK
------------------------------------------------------------
    Model_init()_    -- mode:              autoencoding
    Model_init()_    -- Unsupervised :     True
    Model_init()_    -- Support for unsupervised training  in 'autoencoding' mode is True
    Model_init()_    -- Criterion:         MSELoss()
    Model_init()_    -- use_snnl :         False
    Model_init()_    -- temperature :      0
    Model_init()_    -- temperature LR:    0.0

------------------------------------------------------------
 Building Autoencoder from NOTEBOOK
------------------------------------------------------------
setup_prim_optimizer()
    AE init() -- mode               : autoencoding
    AE init() -- unsupervised       : True
    AE init() -- layer_types        : ['linear', 'relu', 'linear', 'relu', 'linear', 'linear', 'relu', 'linear', 'relu', 'linear']

In [57]:
# display_model_summary(model)

In [58]:
# list_namespace(args)

# Load Model from saved checkpoint 

In [19]:
args.load_checkpoint = utils.load_checkpoint_v5
args.save_checkpoint = utils.save_checkpoint_v5
args.load_checkpoint
args.save_checkpoint
args.ckpt

<function snnl.utils.utils_ptsnnl.load_checkpoint_v5(model, filename, dryrun=False, verbose=False, quiet=False)>

<function snnl.utils.utils_ptsnnl.save_checkpoint_v5(epoch, model, args, filename=None, update_latest=False, update_best=False, verbose=False)>

In [21]:
# epochs = 50
# ex_runmode = 'baseline'
# ex_runmode = 'base'
# ex_date = "20240709"
# ex_time = "1950"
# indicator = 's'
##
ex_runmode = 'snnl'
ex_datetime = "20240709_2235"
indicator = 'd'
##--------------------------------------
ex_runmode = 'base'
ex_datetime = "20240827_1938"
indicator = 's'

suffix = '*'

In [22]:
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_*_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_*.pt"
# print(filename)
# print(file_pattern)

filename = f"{model.name}_{ex_runmode}_{indicator}cpb{COMPOUNDS_PER_BATCH}_{LATENT_DIM}Ltnt_{HIDDEN_1}_{ex_datetime}.pt"
file_pattern = f"{model.name}_{ex_runmode}_{indicator}cpb{COMPOUNDS_PER_BATCH}_{LATENT_DIM}Ltnt_{HIDDEN_1}_{ex_datetime}{suffix}.pt"
print(filename)
print(file_pattern)
print(args.ckpt)

AE_base_scpb200_150Ltnt_512_20240827_1938.pt
AE_base_scpb200_150Ltnt_512_20240827_1938*.pt
None


In [24]:
import glob
# filelist = glob.glob(os.path.join('ckpts',file_pattern))
filelist = glob.glob(file_pattern,root_dir = './ckpts')
filelist = sorted(filelist)
for i , f in enumerate(filelist):
    print(f" {i:3d} :  {f}")
# epochlist =sorted([int(x[-6:-3]) for x in filelist])
# epochlist
# epochlist =[int(x[31:34]) for x in filelist]
# epochlist =[int(x[26:29]) for x in filelist]
# epochlist = [200]
# epochlist = [350]

   0 :  AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt
   1 :  AE_base_scpb200_150Ltnt_512_20240827_1938_LAST_ep_100.pt
   2 :  AE_base_scpb200_150Ltnt_512_20240827_1938_LAST_ep_200.pt
   3 :  AE_base_scpb200_150Ltnt_512_20240827_1938_LAST_ep_400.pt
   4 :  AE_base_scpb200_150Ltnt_512_20240827_1938_ep_100.pt
   5 :  AE_base_scpb200_150Ltnt_512_20240827_1938_ep_200.pt
   6 :  AE_base_scpb200_150Ltnt_512_20240827_1938_ep_300.pt
   7 :  AE_base_scpb200_150Ltnt_512_20240827_1938_ep_400.pt


### Test model loading

In [25]:
args.ckpt = 'AE_snnl_dcpb200_150Ltnt_512_20240709_2235_ep_500.pt'
args.ckpt = filelist[0]

if args.ckpt is not None:
    resume_training = True
    model_copy, last_epoch = args.load_checkpoint(model, args.ckpt, verbose = False)
    model_copy.train()
    model_copy.device = args.current_device
    model_copy = model_copy .cuda(device=args.current_device)
    logging.info(f" Loaded Model device {model_copy.device} -  Last completed epoch : {last_epoch}")
else:
    logging.info(f" ERROR - Checkpoint not found  ")

args.current_device, model.device, model_copy.device
del model_copy

2024-08-28 17:48:19,831 - root - INFO: -  Load model checkpoint from  AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt
2024-08-28 17:48:19,919 - snnl.utils.utils_ptsnnl - INFO: -  Loaded from checkpoint AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt successfully. last epoch on checkpoint: 302
2024-08-28 17:48:19,920 - snnl.utils.utils_ptsnnl - INFO: -  Model best training metric   : 0.470116 - epoch: 5
2024-08-28 17:48:19,921 - snnl.utils.utils_ptsnnl - INFO: -  Model best validation metric : 0.470615 - epoch: 302


Autoencoder(
  (primary_criterion): MSELoss()
  (layers): ModuleList(
    (0): Linear(in_features=1471, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=150, bias=True)
    (5): Linear(in_features=150, out_features=512, bias=True)
    (6): ReLU()
    (7): Linear(in_features=512, out_features=1024, bias=True)
    (8): ReLU()
    (9): Linear(in_features=1024, out_features=1471, bias=True)
  )
)

2024-08-28 17:48:19,936 - root - INFO: -  Loaded Model device cuda:1 -  Last completed epoch : 302


('cuda:1', 'cuda:1', 'cuda:1')

# Run data through model and get latent representation

In [26]:
# = utils.load_model_from_ckpt(model, ex_mode, ex_timestamp, ex_title, epochs, ex_cpb, ex_factor )
output_file_cols =  ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022','Metadata_Hash','Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA' ,'Metadata_Permiation']
output_file_cols += [f'Feature_{x:03d}' for x in range(LATENT_DIM)]

print(len(output_file_cols))
epoch = "ep_500"
# epoch = "BEST" ## "LAST"

161


In [36]:
 
BASELINE_BEST_OUTPUT_FILE  = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_BASELINE_{ex_datetime}_BEST_{{datatype}}.csv"
BASELINE_LAST_OUTPUT_FILE  = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_BASELINE_{ex_datetime}_LAST_{{datatype}}.csv"
BASELINE_OTHER_OUTPUT_FILE = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_BASELINE_{ex_datetime}_{epoch}_{{datatype}}.csv"

SNNL_BEST_OUTPUT_FILE  = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_SNNL_{ex_datetime}_BEST_{{datatype}}.csv"
SNNL_LAST_OUTPUT_FILE  = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_SNNL_{ex_datetime}_LAST_{{datatype}}.csv"
SNNL_OTHER_OUTPUT_FILE = f"3smpl_prfl_embedding_{len(output_file_cols)}_HashOrder_SNNL_{ex_datetime}_{epoch}_{{datatype}}.csv"

print(BASELINE_BEST_OUTPUT_FILE)
print(BASELINE_LAST_OUTPUT_FILE)
print(BASELINE_OTHER_OUTPUT_FILE)
print(SNNL_BEST_OUTPUT_FILE)
print(SNNL_LAST_OUTPUT_FILE)
print(SNNL_OTHER_OUTPUT_FILE)

3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_BEST_{datatype}.csv
3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_LAST_{datatype}.csv
3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_ep_500_{datatype}.csv
3smpl_prfl_embedding_161_HashOrder_SNNL_20240827_1938_BEST_{datatype}.csv
3smpl_prfl_embedding_161_HashOrder_SNNL_20240827_1938_LAST_{datatype}.csv
3smpl_prfl_embedding_161_HashOrder_SNNL_20240827_1938_ep_500_{datatype}.csv


In [37]:
OUTPUT_FILE = BASELINE_BEST_OUTPUT_FILE
# OUTPUT_FILE = SNNL_LAST_OUTPUT_FILE
# OUTPUT_FILE = SNNL_BEST_OUTPUT_FILE
# OUTPUT_FILE = SNNL_OTHER_OUTPUT_FILE

for datatype in ['train', 'test']:
    print(os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype=datatype)))


/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_BEST_train.csv
/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_BEST_test.csv


In [38]:
filename = filelist[0]
# filename = 'AE_snnl_dcpb200_150Ltnt_512_20240709_2235_ep_500.pt'
print(filename)

AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt


In [39]:
model, last_epoch = utils.load_model_from_ckpt(model, args, filename = filename, cuda_device = model.device, verbose = False)
if model is None:
    print(f"model is none")
else:
    print(f"model loaded - last epoch is {last_epoch}")

2024-08-28 17:54:17,972 - root - INFO: -  Load model checkpoint from  AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt
2024-08-28 17:54:18,048 - snnl.utils.utils_ptsnnl - INFO: -  Loaded from checkpoint AE_base_scpb200_150Ltnt_512_20240827_1938_BEST.pt successfully. last epoch on checkpoint: 302
2024-08-28 17:54:18,049 - snnl.utils.utils_ptsnnl - INFO: -  Model best training metric   : 0.470116 - epoch: 5
2024-08-28 17:54:18,050 - snnl.utils.utils_ptsnnl - INFO: -  Model best validation metric : 0.470615 - epoch: 302
2024-08-28 17:54:18,053 - snnl.utils.utils_ptsnnl - INFO: -  Loaded model device     : cuda:1


model loaded - last epoch is 302


# Get latent representations of Training and Test data

In [40]:

for datatype in ['train', 'test']:
    print(f"****************  {datatype} - get latent representation ***************")
    write_header = True
    mdl_outputs = utils.get_latent_representation(model, data_loader[datatype], embedding_layer = 4, verbose = False)

    print(f"****************  {datatype} - convert to dataframe  ***************")
    df_output = pd.DataFrame.from_records(mdl_outputs, columns = output_file_cols)

    print(f"****************  {datatype} - write to csv file : {OUTPUT_FILE.format(datatype=datatype)} ***************")
    df_output.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype=datatype)), mode = 'a', index = False, header = write_header)
    write_header = False


****************  train - get latent representation ***************
 embedding layer: 4 - model.layer: in: 512 out: 150
(600, 161) (600, 161)
(600, 161) (1200, 161)
(600, 161) (1800, 161)
(600, 161) (2400, 161)
(600, 161) (3000, 161)
(600, 161) (3600, 161)
(600, 161) (4200, 161)
(600, 161) (4800, 161)
(600, 161) (5400, 161)
(600, 161) (6000, 161)
(600, 161) (6600, 161)
(600, 161) (7200, 161)
(600, 161) (7800, 161)
(600, 161) (8400, 161)
(600, 161) (9000, 161)
(600, 161) (9600, 161)
(600, 161) (10200, 161)
(600, 161) (10800, 161)
(600, 161) (11400, 161)
(600, 161) (12000, 161)
(600, 161) (12600, 161)
(600, 161) (13200, 161)
(600, 161) (13800, 161)
(600, 161) (14400, 161)
(600, 161) (15000, 161)
(600, 161) (15600, 161)
(600, 161) (16200, 161)
(600, 161) (16800, 161)
(600, 161) (17400, 161)
(600, 161) (18000, 161)
(600, 161) (18600, 161)
(600, 161) (19200, 161)
(600, 161) (19800, 161)
(600, 161) (20400, 161)
(600, 161) (21000, 161)
(600, 161) (21600, 161)
(600, 161) (22200, 161)
(600, 161

# Read Embedded Features CSV file

In [41]:
INPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype='test'))
INPUT_FILE

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_BEST_test.csv'

In [46]:
metadata_cols = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022','Metadata_Hash','Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
# df_input = pd.read_csv(INPUT_FILE,usecols=['Metadata_Source'])
df_input = pd.read_csv(INPUT_FILE,usecols=metadata_cols)
# df_input = pd.read_csv(INPUT_FILE,num)

In [47]:
df_input.info()
df_input.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34542 entries, 0 to 34541
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Metadata_Source      34542 non-null  object 
 1   Metadata_Batch       34542 non-null  object 
 2   Metadata_Plate       34542 non-null  object 
 3   Metadata_Well        34542 non-null  object 
 4   Metadata_JCP2022     34542 non-null  object 
 5   Metadata_Hash        34542 non-null  int64  
 6   Metadata_Bin         34542 non-null  int64  
 7   Metadata_TPSA        34542 non-null  float64
 8   Metadata_lnTPSA      34542 non-null  float64
 9   Metadata_log10TPSA   34542 non-null  float64
 10  Metadata_Permiation  34542 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 2.9+ MB


(34542, 11)

In [48]:
df_input

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,Metadata_Permiation
0,source_11,Batch2,EC000046,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
1,source_11,Batch2,EC000063,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
2,source_9,20211013-Run14,GR00003281,J15,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
3,source_5,JUMPCPE-20210908-Run28_20210909_072022,AETJUM107,J10,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,0.0
4,source_11,Batch3,EC000139,G04,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,0.0
...,...,...,...,...,...,...,...,...,...,...,...
34537,source_9,20211103-Run16,GR00004412,B19,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,0.0
34538,source_9,20211103-Run16,GR00004410,R43,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,0.0
34539,source_6,p211109CPU2OS48hw384exp035JUMP,110000296165,N13,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,0.0
34540,source_1,Batch4_20221012,UL000093,AB26,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,0.0


In [52]:
y_label = df_input.Metadata_Permiation.astype(np.int8)
bin_cnt = np.bincount(y_label)
bin_cnt

array([30378,  4164])

In [53]:
INPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype='train'))
INPUT_FILE

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_BASELINE_20240827_1938_BEST_train.csv'

In [54]:
metadata_cols = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022','Metadata_Hash','Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
# df_input = pd.read_csv(INPUT_FILE,usecols=['Metadata_Source'])
df_train = pd.read_csv(INPUT_FILE,usecols=metadata_cols)

In [56]:
df_train.shape
df_train.info()

(312000, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Metadata_Source      312000 non-null  object 
 1   Metadata_Batch       312000 non-null  object 
 2   Metadata_Plate       312000 non-null  object 
 3   Metadata_Well        312000 non-null  object 
 4   Metadata_JCP2022     312000 non-null  object 
 5   Metadata_Hash        312000 non-null  int64  
 6   Metadata_Bin         312000 non-null  int64  
 7   Metadata_TPSA        312000 non-null  float64
 8   Metadata_lnTPSA      312000 non-null  float64
 9   Metadata_log10TPSA   312000 non-null  float64
 10  Metadata_Permiation  312000 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 26.2+ MB


In [57]:
df_train

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,Metadata_Permiation
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,0.0
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,0.0
...,...,...,...,...,...,...,...,...,...,...,...
311995,source_6,p211004CPU2OS48hw384exp031JUMP,110000296294,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,0.0
311996,source_2,20210823_Batch_10,1086292051,L21,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,0.0
311997,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143006,A07,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,0.0
311998,source_6,p210928CPU2OS48hw384exp030JUMP,110000296370,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,0.0


In [59]:
y_label = df_train.Metadata_Permiation.astype(np.int8)
bin_cnt = np.bincount(y_label)
bin_cnt

array([274836,  37164])

In [54]:
312000+34542

346542

In [58]:
del df_tmp, df_input

# Load checkpoint

In [78]:
# filename = "AE_snnl_dcpb200_150Ltnt_512_20240709_2235_BEST_404.pt"
# filename = "AE_snnl_dcpb200_150Ltnt_512_20240709_2235_BEST_1.pt"
# filename = "AE_snnl_dcpb200_150Ltnt_512_20240709_2235_LAST_ep_700.pt"
# filename = "AE_snnl_dcpb200_150Ltnt_512_240709_2235_LAST_ep_705.pt"
# filename = args.ckpt

In [96]:
idxs = [3]
filenames = [filelist[idx] for idx in idxs]
for idx, filename in enumerate(filenames):
    print(filename)
    epoch = -1
    if filename[-3:] != '.pt':
        filename+='.pt'
    ckpt_file = os.path.join("ckpts", filename)
    # print(f" Checkpoint file : {ckpt_file}")
    
    try:
        checkpoint = torch.load(ckpt_file)
    except FileNotFoundError:
        Exception("Previous state checkpoint not found.")
        print("FileNotFound Exception")
    except :
        print("Other Exception")
        print(sys.exc_info())
    
    # checkpoint.keys()
    
    print(f"{idx:3d} {filename:60s} -   epoch: {checkpoint['epoch']:4d}          {checkpoint.keys()}")

AE_snnl_dcpb200_150Ltnt_512_20240709_2235_LAST_ep_700.pt
  0 AE_snnl_dcpb200_150Ltnt_512_20240709_2235_LAST_ep_700.pt     -   epoch:  700          dict_keys(['epoch', 'state_dict', 'optimizers', 'optimizers_state_dict', 'schedulers', 'schedulers_state_dict', 'params'])


In [97]:
for k,v in checkpoint.items():
    print(f" {k:30}  tpye: {type(v)}")

 epoch                           tpye: <class 'int'>
 state_dict                      tpye: <class 'collections.OrderedDict'>
 optimizers                      tpye: <class 'dict'>
 optimizers_state_dict           tpye: <class 'dict'>
 schedulers                      tpye: <class 'dict'>
 schedulers_state_dict           tpye: <class 'dict'>
 params                          tpye: <class 'dict'>


In [109]:
# for k,v in checkpoint['optimizers'].items():
    # print(f" {k:30}  tpye: {type(v)}")
checkpoint['optimizers']['prim']
for k,v in checkpoint['optimizers_state_dict'].items():
    print(f" {k:30}  tpye: {type(v)}")
    print(v)


 prim                            tpye: <class 'dict'>
{'state': {0: {'step': tensor(280000.), 'exp_avg': tensor([[ 7.2271e-07,  1.2926e-06, -1.2854e-06,  ..., -4.4894e-07,  1.0065e-06, -7.0389e-08],
        [-3.6944e-07,  1.1674e-07,  3.3869e-07,  ...,  5.4228e-07, -2.2068e-06, -1.6183e-07],
        [ 1.8231e-06,  2.4441e-06,  2.0402e-06,  ..., -2.4858e-06, -2.6043e-06, -1.5995e-06],
        ...,
        [-4.1211e-06, -1.0590e-06, -1.9245e-06,  ...,  3.0483e-06,  1.2641e-06,  3.2228e-06],
        [-5.2276e-07, -6.2790e-08,  1.6012e-06,  ...,  4.3297e-06, -1.5604e-06,  4.5590e-06],
        [-1.8927e-06, -2.1949e-06, -1.5110e-06,  ..., -6.5841e-07,  2.0136e-06, -1.1450e-06]], device='cuda:0'), 'exp_avg_sq': tensor([[4.3622e-11, 1.8160e-11, 1.8048e-11,  ..., 8.8139e-11, 1.0911e-10, 9.8168e-11],
        [1.0076e-10, 2.9558e-11, 2.5882e-11,  ..., 1.3456e-10, 1.6557e-10, 1.2977e-10],
        [6.8379e-11, 3.5418e-11, 3.9468e-11,  ..., 1.8371e-10, 2.4346e-10, 1.9584e-10],
        ...,
        

In [114]:
# for k,v in checkpoint['schedulers'].items():
    # print(f" {k:30}  tpye: {type(v)}")    
print(checkpoint['schedulers']['prim'].__dict__)
checkpoint['schedulers_state_dict']['prim'] 


{'factor': 0.5, 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
), 'min_lrs': [0], 'patience': 50, 'verbose': False, 'cooldown': 10, 'cooldown_counter': 0, 'mode': 'min', 'threshold': 1e-06, 'threshold_mode': 'rel', 'best': 0.0886982049793005, 'num_bad_epochs': 0, 'mode_worse': inf, 'eps': 1e-08, 'last_epoch': 700, '_last_lr': [0.0001]}


{'factor': 0.5,
 'min_lrs': [0],
 'patience': 50,
 'verbose': False,
 'cooldown': 10,
 'cooldown_counter': 0,
 'mode': 'min',
 'threshold': 1e-06,
 'threshold_mode': 'rel',
 'best': 0.0886982049793005,
 'num_bad_epochs': 0,
 'mode_worse': inf,
 'eps': 1e-08,
 'last_epoch': 700,
 '_last_lr': [0.0001]}

In [101]:
# model_attributes = checkpoint['model'].__dict__
model_attributes = checkpoint['params']

for k,v in model_attributes.items():
    if type(v) in [dict]
    print(f" {k:30}  tpye: {str(type(v)):25s}   values: {v if type(v) is not dict else '--'} ")

 name                            tpye: <class 'str'>               values: AE 
 layer_types                     tpye: <class 'list'>              values: ['linear', 'relu', 'linear', 'relu', 'linear', 'linear', 'relu', 'linear', 'relu', 'linear'] 
 non_linearities                 tpye: <class 'list'>              values: ['relu', 'relu', 'relu', 'relu'] 
 monitor_grads_layer             tpye: <class 'NoneType'>          values: None 
 use_single_loss                 tpye: <class 'bool'>              values: False 
 optimizer                       tpye: <class 'NoneType'>          values: None 
 scheduler                       tpye: <class 'NoneType'>          values: None 
 learning_rate                   tpye: <class 'float'>             values: 0.0001 
 use_prim_optimizer              tpye: <class 'bool'>              values: True 
 use_prim_scheduler              tpye: <class 'bool'>              values: True 
 use_annealing                   tpye: <class 'bool'>              values

In [68]:
checkpoint['state_dict'].keys()

KeyError: 'state_dict'

In [ ]:
for key, value in checkpoint.items():
    logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")

if dryrun:
    for key, value in checkpoint['params'].items():
        logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")
else:
    model.load_state_dict(checkpoint['state_dict'])
    # model = checkpoint['state_dict']
    model.optimizers = {k:v for k,v in checkpoint['optimizers'].items()}
    model.schedulers = {k:v for k,v in checkpoint['schedulers'].items()}

    for key, value in checkpoint['params'].items():
        logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")
        model.__dict__[key] = value     
    

In [ ]:
    epoch = -1
    if filename[-3:] != '.pt':
        filename+='.pt'
    logging.info(f" Load model checkpoint from  {filename}")    
    ckpt_file = os.path.join("ckpts", filename)
    
    try:
        checkpoint = torch.load(ckpt_file)
    except FileNotFoundError:
        Exception("Previous state checkpoint not found.")
        print("FileNotFound Exception")
    except :
        print("Other Exception")
        print(sys.exc_info())

    for key, value in checkpoint.items():
        logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")
    
    if dryrun:
        for key, value in checkpoint['params'].items():
            logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")
    else:
        model.load_state_dict(checkpoint['state_dict'])
        # model = checkpoint['state_dict']
        model.optimizers = {k:v for k,v in checkpoint['optimizers'].items()}
        model.schedulers = {k:v for k,v in checkpoint['schedulers'].items()}

        for key, value in checkpoint['params'].items():
            logging.debug(f"{key:40s}, {str(type(value)):60s}  -- model attr set")
            model.__dict__[key] = value     
        

In [42]:

def init_resume_training(model, args):
    if args.ckpt is not None:
        model.resume_training = True
        model , last_epoch = args.load_checkpoint(model, args.ckpt, verbose = True )  
        print(last_epoch, type(model))
        print(model)
        model.train()
        model.device = args.current_device
        model = model.to(args.current_device)
        # model = model.cuda(device=current_device)
        logging.info(f" Loaded Model device {model.device} -  Last completed epoch : {last_epoch}")
        model.starting_epoch = last_epoch
        model.ending_epoch = last_epoch + args.epochs
        logging.info(f" RESUME TRAINING - Run {args.epochs} epochs: epoch {model.starting_epoch+1} to {model.ending_epoch} ")

        if 'gen' not in model.training_history:
            print('*' * 60)
            print(f" Define self.training_history['gen'] ")
            print('*' * 60)
            model.training_history['gen'] = {'trn_best_metric' : 0, 'trn_best_metric_ep' : 0, 'trn_best_loss': np.inf, 'trn_best_loss_ep' : 0 ,
                                            'val_best_metric' : 0, 'val_best_metric_ep' : 0, 'val_best_loss': np.inf, 'val_best_loss_ep' : 0 }        
        
            for key in ['trn', 'val']:
                tmp = np.argmin(model.training_history[key][f'{key}_ttl_loss'])
                model.training_history['gen'][f'{key}_best_loss_ep'] = tmp
                model.training_history['gen'][f'{key}_best_loss']    = model.training_history[key][f'{key}_ttl_loss'][tmp]
                
                tmp1 = np.argmax(model.training_history[key][f'{key}_R2_score'])
                model.training_history['gen'][f'{key}_best_metric_ep'] = tmp1
                model.training_history['gen'][f'{key}_best_metric'] = model.training_history[key][f'{key}_R2_score'][tmp1]
        
    else:
        model.resume_training = False
        model.starting_epoch = 0
        model.ending_epoch = args.epochs
        logging.info(f" INITIALIZE TRAINING - Run {args.epochs} epochs: epoch {model.starting_epoch+1} to {model.ending_epoch} ")

    model.best_metric = model.training_history['gen'][f'val_best_metric']  
    model.best_epoch  = model.training_history['gen'][f'val_best_metric_ep']  
    return model